# 多模态提示词
多模态模型能够同时理解、处理、生成文字、图像、语音、视频等多种信息，多模态模型大致分为理解型模型和生成类模型，其中理解型模型输入多模态信息，输出是文字类型，而生成类模型输入可以是多模态信息，输出也是多模态类，比如让模型生成一张“阳光下的彩虹”的图像，多模态生成类模型就能够生成精美的图像。

简单来说，单模态模型仅有文字，相当于人类仅能对话，看不到图像、听不到声音、说不出话，多模态模型相当于赋予模型人类的五感，它不仅能够去认识我们的这个文字，还能听懂我们的语音，看到我们的图片，理解我们的视频。

本节我们将针对多模态模型设计提示词，当输入是由文字、图像、视频、音频等多种模态组成的信息时，如何引导AI生成我们理想的结果。

使用的模型为Qwen系列模型，其中图像和视频都由Qwen2.5-vl模型实现。关于推理代码，我们可以从[modelscope官网](https://www.modelscope.cn/models/Qwen/Qwen2.5-VL-7B-Instruct)得知。

In [1]:
### 模型加载
from modelscope import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "/home/lixinyu/weights/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("/home/lixinyu/weights/Qwen2.5-VL-3B-Instruct")

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.28it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor conf

In [2]:
# 推理代码
from typing import List, Union, Dict, Any
import torch
from modelscope import Qwen2_5_VLForConditionalGeneration,AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

def qwen_vl_chat(
    model: Qwen2_5_VLForConditionalGeneration,
    processor: AutoProcessor,
    text_query: str,
    images: Union[str, List[str]] = None,     # 一张图或多张图
    videos: Union[str, List[str], List[List[str]]] = None,   # 视频路径/URL 或帧列表
    fps: float = 1.0,
    max_pixels: int = None,   # 例如 360*420，可控制显存
    max_new_tokens: int = 512,
    temperature: float = 0.9,
    device: str = "cuda"
) -> str:
    """
    统一接口：图像+文字 或 视频+文字 均可
    """
    content = [{"type": "text", "text": text_query}]

    # 1) 处理图像
    if images is not None:
        if isinstance(images, str):
            images = [images]
        for img in images:
            content.insert(0, {"type": "image", "image": img})

    # 2) 处理视频
    if videos is not None:
        if isinstance(videos, str) or (isinstance(videos, list) and len(videos) and isinstance(videos[0], str)):
            # 单个路径/URL 或 帧列表
            content.insert(0, {"type": "video", "video": videos})
        else:
            raise ValueError("videos 仅支持 str 或 List[str]/List[List[str]] 格式")

    messages = [{"role": "user", "content": content}]

    # 3) 构造输入
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs, video_kwargs = process_vision_info(
        messages, return_video_kwargs=True
    )

    common_kwargs = {"fps": fps} if videos is not None else {}
    if max_pixels is not None:
        common_kwargs["max_pixels"] = max_pixels

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
        **common_kwargs,
        **video_kwargs
    ).to(device)

    # 4) 推理
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True
        )
    generated_ids = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    response = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    return response

## 图像理解
对于多模态模型，提示词的设计需要注意下面几点：
1. 准确清楚
2. 必要情况下，对图像进行标记

In [3]:
image_path="./picture/fapiao1.png"
text="请提取其中所有的信息，并按照json格式输出"

In [4]:
print(qwen_vl_chat(model,processor,text_query=text,images=[image_path]))

```json
{
    "公司名称": "中科视拓（南京）科技有限公司",
    "税务登记号": "91320191MA1XM5TX71",
    "发票编号": "24344",
    "开票日期": "2024年01月02日",
    "购买方信息": {
        "名称": "xxxxxx限公司",
        "统一社会信用代码/纳税人识别号": ""
    },
    "销售方信息": {
        "名称": "中科视拓（南京）科技有限公司",
        "统一社会信用代码/纳税人识别号": "91320191MA1XM5TX71"
    },
    "项目名称": "*信息技术服务*云服务器服务费",
    "规格型号": "项",
    "单位": "1 4716.98113207547",
    "数量": "",
    "单价": "4716.98",
    "金额": "￥4716.98",
    "税率/征收率": "6%",
    "税额": "283.02",
    "合计": "伍仟圆整",
    "价税合计(大写)": "(小写)¥ 5000.00",
    "备注": "购方开户银行:银行账号:127；\n销方开户银行:支行;银行账号:7",
    "开票人": "卢朦"
}
```


我们也可以更精准点

In [5]:
image_path="./picture/fapiao1.png"
text="请提取发票中的给出的合计的金额，并按照json格式输出"

In [6]:
print(qwen_vl_chat(model,processor,text_query=text,images=[image_path]))

```json
{
  "合计": "￥4716.98"
}
```


我们也可以多张图分析

In [9]:
images=["./picture/banma.jpeg","./picture/xiongmao.jpeg","./picture/xiaoxiongmao.jpeg"]
text="请分析三张图之间的相同点，不同点"

In [10]:
print(qwen_vl_chat(model,processor,text_query=text,images=images))

这三个图分别是熊猫和小熊猫的图片，还有斑马的照片，所以都是动物图片。
不同点：熊猫主要分布在大熊猫保护区和动物园，而小熊猫也分布在熊猫的保护区，但是它们体型相差较大。斑马是生活在非洲草原的一种动物，属于马科物种。
相同点：这三张图片都展示了动物行为和生活环境中的有趣瞬间，给观众留下深刻的印象。


如果图像中元素过多，可以对我们需要询问的对象单独框出，然后让模型理解问题，比如：

In [11]:
image_path="./picture/damoxing1.png"
text="请你解释红框中的原理"

In [12]:
print(qwen_vl_chat(model,processor,text_query=text,images=[image_path]))

红框中的原理是脑科学中的“存储与检索”概念，用于描述大脑如何处理、储存和检索信息。这一过程可以分解为以下几个主要步骤：

1. **获取（Grasp）**：接收到环境输入信号。
2. **编码（Code）**：将输入以适宜的方式编码（如特征提取、模型构建等）。
3. **储存（Store）**：将编码后的信息储存在记忆中（如长期记忆或短期记忆）。
4. **检索（Retrieve）**：需要时从记忆中检索相关信息。
5. **修改（Modify）**：对检索出的信息进行分析、调整。

在视觉场景中，如图像识别任务，这个过程表现为神经网络识别输入图像并转换成模式表示，如神经元活跃状态图（neural activity map）、特征图（feature map），然后将其储存在记忆中，以便于后续使用。

这种过程体现了智能系统的学习能力，即通过不断经历、观察（Encoding and Grasp）、处理存储数据，并根据新获得的数据进行优化（Learn, Summary, Recalling）。这些过程中蕴含了强大的学习、推理、回忆、计划等高级功能，这也是理解人工智能实现基础。


## 视频分析
多模态模型也可以用来分析视频，不过在实际使用的时候，由于视频需要的资源都比较多，因此本地推理时显存占用一般承受不起，不过，如果资源充足，运行下面的代码即可：

```python
print(qwen_vl_chat(model,processor,text_query=text,videos=[video_path]))
```

所以本节我们使用API接口实现，我们使用阿里云百炼平台，通过调用视觉模型的API接口在本地做一个demo：

In [ ]:
from openai import OpenAI

def qwen_video_chat(video_url: str, text: str,
                    model: str = "qwen-vl-max-latest",
                    max_tokens: int = 512,
                    temperature: float = 0.7) -> str:
    """
    调用阿里百炼「qwen-vl-max-latest」模型，对给定视频进行问答。

    参数
    ----
    video_url : str
        视频地址（http/https 或 file://）
    text : str
        用户想问的问题
    model / max_tokens / temperature :
        生成参数，可按需调整

    返回
    ----
    answer : str
        模型回答文本
    """
    client = OpenAI(
        api_key="sk-",          # 换成自己的 Key
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )

    try:
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": [{"type": "text", "text": "You are a helpful assistant."}]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "video_url",
                            "video_url": {"url": video_url}
                        },
                        {
                            "type": "text",
                            "text": text
                        }
                    ]
                }
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return completion.choices[0].message.content

    except Exception as e:
        # 捕获网络或权限异常，给出友好提示
        return (f"请求失败，可能原因：\n"
                f"1. 网络不通或代理设置问题；\n"
                f"2. 视频链接无效（404/403/超时）；\n"
                f"3. 密钥/计费异常。\n"
                f"建议检查链接合法性、网络连通性或稍后重试。\n"
                f"错误详情：{e}")

In [14]:
video = "https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241115/cqqkru/1.mp4"
question = "这段视频的内容是什么？"

In [15]:
print(qwen_video_chat(video, question))

这段视频展示了一位年轻女性的特写镜头。她有着短发，面带微笑，看起来非常开心和友好。她的穿着是一件粉色的针织开衫搭配白色的内搭，整体风格显得很清新自然。背景模糊，但可以看出是在户外，可能是一个校园或公园等地方。视频中的女性表情生动，笑容灿烂，给人一种温暖和愉快的感觉。右上角有“通义·AI合成”的字样，表明这段视频可能是通过AI技术合成的。
